### Import Libraries

In [4]:
import pandas as pd
from sklearn.cluster import KMeans

### Get Data From Wikipedia
After getting the data from wikipedia, the columns were consolidated into a single column.

In [5]:
raw_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
raw_data = raw_data.unstack().reset_index(drop=True)
raw_data

0                                                    M1A
1                                                    M2A
2                                                    M3A
3                                                    M4A
4                                                    M5A
                             ...                        
535                                                  NaN
536                                                  NaN
537                                                  NaN
538    Mimico NW / The Queensway West / South of Bloo...
539                                                  NaN
Length: 540, dtype: object

### Create Dataframe

In [6]:
# create dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
df = pd.DataFrame(columns=column_names)

for i, row in raw_data.iteritems():
    postal_code = raw_data[i][:3]
    if "(" not in row:
        borough = raw_data[i][3:]
        neighborhood = ""
    else:
        places = raw_data[i][3:].split('(', 1)
        borough = places[0]
        neighborhood = places[1].replace(')', '').replace(' /', ',')
    df = df.append({'PostalCode': postal_code, 'Borough': borough, 'Neighborhood': neighborhood}, ignore_index=True)

# Remove rows with Not assigned Boroughs
df.drop(df[df['Borough']== 'Not assigned'].index , inplace=True)

df.head()

TypeError: 'float' object is not subscriptable

### Get data shape

In [ ]:
df.shape

### Read neighborhood geographical coordinates

In [ ]:
coords = pd.read_csv('Geospatial_Coordinates.csv', names=['PostalCode', 'Latitude', 'Longitude'])
coords

### Adding the coordinates to the initial dataframe

In [ ]:
final_df = pd.merge(df, coords, on='PostalCode')
final_df

In [ ]:
final_df['Borough'].unique()

### Create initial map

In [ ]:
import folium

In [ ]:
toronto_map = folium.Map(location=[43.6532, -79.3832], zoom_start=11)
toronto_map

In [ ]:
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### Get data that has "Toronto" in Borough

In [ ]:
toronto_data = final_df[final_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

In [ ]:
# create map of Manhattan using latitude and longitude values
borough_map = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(borough_map)  
    
borough_map

### Setup Foursquare credentials

In [ ]:
CLIENT_ID = 'DGNQU5N1GZ1HZQ1UJQRSUDUYX2ZOHVRZHIJI2T5V3IHKVACG'
CLIENT_SECRET = 'M3YKN5GUP314DKXSIYCFK5ONAPYMYHUGAMSNKH4EVQIQSGMM'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
toronto_data.loc[0, 'Neighborhood']

In [ ]:
borough_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = toronto_data.loc[0, 'Neighborhood'] # borough name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    500, 
    100)
url # display URL

In [ ]:
import requests
results = requests.get(url).json()
results